In [1444]:
#read data
#test 1 - "Project-2_Test_Data/Data-1_test.csv"
import pandas as pd
train_file_path = "Project-2_Test_Data/Data-2_test.csv"
train_data = pd.read_csv(train_file_path)
train_data.head()

,example_id,text,aspect_term,term_location
0,32933228#1700177#1_2,I reccomend the fried pork dumplings[comma] th...,fried rice,71--81
1,35820984#608922#3_0,The staff is very sharp and they look good too.,staff,4--9
2,35170181#0#5_1,The best dessert[comma] a chocolate and peanut...,chocolate and peanut butter tart,20--52
3,33067279#1612676#1_1,The food was very good and I was pleasantly su...,vegan options,69--82
4,32882616#562969#3_0,I never had an orange donut before so I gave i...,orange donut,15--27


In [1445]:
train_data.shape

(1120, 4)

In [1446]:
class_value = train_data[' class']
class_value.describe()
Y = class_value

KeyError: ' class'

In [1440]:
Y.shape

(3602,)

In [1344]:
class_value.head()

 1    2164
-1     805
 0     633
Name:  class, dtype: int64

In [1447]:

example_ids = train_data['example_id']
X = train_data.drop(['example_id'," text"],axis=1)
X.head()

,aspect_term,term_location
0,fried rice,71--81
1,staff,4--9
2,chocolate and peanut butter tart,20--52
3,vegan options,69--82
4,orange donut,15--27


In [1448]:
#X.shape[0]

In [956]:
#training bigram tagger

from nltk.corpus import brown

brown_tagged_sents = brown.tagged_sents()
size = int(len(brown_tagged_sents) * 0.9)

train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]

In [143]:
import nltk
def build_backoff_tagger(train_sents):
    t0 = nltk.DefaultTagger('NN')
    t1 = nltk.UnigramTagger(train_sents, backoff=t0)
    t2 = nltk.BigramTagger(train_sents, backoff=t1)
    return t2

In [144]:
ngram_tagger = build_backoff_tagger(train_sents)

In [145]:
ngram_tagger.evaluate(test_sents)

0.9125751765470128

In [1449]:
text_data = train_data[" text"]
new_text=[]
for idx,text_record in enumerate(text_data):
    aspect_loc = train_data.loc[idx," term_location"]
    locations = aspect_loc.split("--")
    current_text = text_data[idx].replace("[comma]", ",")
    #current_text = current_text[:int(locations[0])] + "<aspect>" + current_text[int(locations[1]):]
    new_text.append([current_text])

In [1450]:

new_text_aspect=[]
for idx,text_record in enumerate(new_text):
    aspect_loc = train_data.loc[idx," term_location"]
    locations = aspect_loc.split("--")
    current_text = text_record[0]
    current_text = current_text[:int(locations[0])] + "<aspect>" + current_text[int(locations[1]):]
    new_text_aspect.append(current_text)

In [1451]:
#extracting words in context 0f 5 from target term
import nltk
from nltk import word_tokenize
from nltk.util import ngrams

sent_list=[]

remove_list = [",",".",".)","..)",").","?","!"]
    
    
for idx,text_word in enumerate(new_text_aspect):
    locations = train_data.loc[idx," term_location"].split("--")
    sent = text_word
    sent_left = sent[:int(locations[0])]
    #token_left = nltk.word_tokenize(sent_left)
    #print(token_left)
    #bigram_left = nltk.bigrams(token_left)
    #bigram_left = *map(' '.join, bigrm)
    sent_right = sent[int(locations[0])+8:]
    #print(sent_right)
    
    #token_right = nltk.word_tokenize(sent_right)
    if sent_right not in remove_list:
        sent_surf = ' '.join(sent_left.split()[-5:])+' '+' '.join(sent_right.split()[:5])
    else:
        sent_surf = ' '.join(sent_left.split()[-5:])
    #print(sent_surf)
    sent_list.append([sent_surf])
    #bigrm_right = [b for b in nltk.bigrams(token_right)]
    #bigrm_right = *map(' '.join, bigrm)
    #print(bigrm_right)
    #sent_left_words = ngrams(token_left,2)
    #sent_right_words = ngrams(sent_right,1)
    #words = sent_left_words[-3:] + sent_right_words[:3]
    #context_word.append(words)
    #print(sent_left_words)
#print(context_word)
#print(sent_list)

In [1452]:
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer, WordPunctTokenizer
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures

features=[]

stops = set(stopwords.words('english'))
for line in sent_list:
    #tokenizer = WordPunctTokenizer()
    tokens = nltk.word_tokenize(line[0])
    #stemmer = PorterStemmer()
    bigram_finder = BigramCollocationFinder.from_words(tokens)
    bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 500)
    #bigrams = ngrams(line,2)
    for bigram_tuple in bigrams:
        x = "%s %s" % bigram_tuple
        tokens.append(x)
    
    #result = [b for b in nltk.bigrams(line) if b[0] not in stops and b[1] not in stops]

    result = [' '.join([w for w in x.split()]) for x in tokens if x.lower() not in stops and len(x) > 8]
    features.append(result)

In [1453]:
feature_main=[]
for f in features:
    if f:
        feature_main.append(f)
        
        
vocab_list=[]
for f in features:
    for ele in f:
        vocab_list.append(ele)
        
        


vocab_list_set = set(vocab_list)

In [1454]:
features_uni=[]
stop_list = stopwords.words('english') +['.',',',')','?','!','(','-',':','The','I','It','We',"'",'They','My','But','us']

stops = set(stop_list)
for line in sent_list:
    tokenizer = WordPunctTokenizer()
    tokens = nltk.word_tokenize(line[0])
    #print(tokens)
    #stemmer = PorterStemmer()
    #bigram_finder = BigramCollocationFinder.from_words(tokens)
    #bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 500)
    #bigrams = ngrams(line,2)
    #for bigram_tuple in bigrams:
    #    x = "%s %s" % bigram_tuple
    #    tokens.append(x)
    
    #result = [b for b in nltk.bigrams(line) if b[0] not in stops and b[1] not in stops]
    new_token = []
    for token in tokens:
        if token not in stops:
            new_token.append(token)
    features_uni.append(new_token)
    #if tokens not in stops:
    #    features_uni.append(result)

In [1455]:
vocab_list_uni=[]
for f in features_uni:
    for ele in f:
        vocab_list_uni.append(ele)
        
        


vocab_list_set_uni = set(vocab_list_uni)

In [1456]:
#features_uni
#vocab_list_uni

In [1457]:
text_df = pd.DataFrame({'text':new_text_aspect})
#print (text_df)

In [1458]:
#uni_df = pd.DataFrame(columns =vocab_list_set_uni )
import numpy as np
uni_df = pd.DataFrame(0, index=np.arange(X.shape[0]), columns =vocab_list_set_uni)


In [1459]:
uni_df.shape

(1120, 1474)

In [1460]:
#bag of words
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(vocabulary=vocab_list_set)
X_train_bag = count_vect.fit_transform(text_df["text"])
X_term_freq = pd.DataFrame(X_train_bag.toarray())

In [1461]:
#dropping columns with sum less than 25
cols_with_less_than_25 = [col for col in X_term_freq.columns 
                                 if abs(X_term_freq[col].sum()) < 1]
reduced_X_train = X_term_freq.drop(cols_with_less_than_25, axis=1)
#reduced_X_val  = val_X.drop(cols_with_missing, axis=1)
#print("Mean Absolute Error from dropping columns with Missing Values:")
#print(getScore(reduced_X_train, train_Y,reduced_X_val,  val_Y))

In [1462]:
#dropping columns with stop words
stop_list = stopwords.words('english')
cols_with_stop_words = [col for col in reduced_X_train.columns 
                                 if reduced_X_train.columns in stop_list ]
reduced_X_train_without_stops = reduced_X_train.drop(cols_with_stop_words, axis=1)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:3846: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = op(self.values, np.asarray(other))


In [1463]:
#reduced_X_train_without_stops.shape

In [1464]:
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer, WordPunctTokenizer
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures
features_uni_tagged=[]
#stop_list = stopwords.words('english') +['.',',',')','?','!','(','-',':','The','I','It','We',"'",'They','My','But','us']

#stops = set(stop_list)
for line in sent_list:
    tokenizer = WordPunctTokenizer()
    tokens = nltk.word_tokenize(line[0])
    #print(tokens)
    #stemmer = PorterStemmer()
    #bigram_finder = BigramCollocationFinder.from_words(tokens)
    #bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 500)
    #bigrams = ngrams(line,2)
    #for bigram_tuple in bigrams:
    #    x = "%s %s" % bigram_tuple
    #    tokens.append(x)
    tagged_tokens = ngram_tagger.tag(tokens)
    #result = [b for b in nltk.bigrams(line) if b[0] not in stops and b[1] not in stops]
    #new_token = []
    #for token in tokens:
        #if token not in stops:
     #       new_token.append(token)
    features_uni_tagged.append(tagged_tokens)

In [1465]:
new_tagged_Sent=[]
for feature in features_uni_tagged:
    sent = ""
    for tuples in feature:
        sent = sent +" " +tuples[0]+"_"+tuples[1]
    
    new_tagged_Sent.append(sent.strip())

In [1466]:
text_df_tagged = pd.DataFrame({'text_tagged':new_tagged_Sent})

In [1467]:
#text_df_tagged.head()

In [1468]:
#bag of words with pos tage
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_bag_tagged = count_vect.fit_transform(text_df_tagged["text_tagged"])
X_term_freq_tagged = pd.DataFrame(X_train_bag_tagged.toarray(),columns=count_vect.get_feature_names())

In [1469]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_bag_tagged)
#X_train_tfidf.shape
X_tfidf_tagged = pd.DataFrame(X_train_tfidf.toarray(),columns=count_vect.get_feature_names())

In [1470]:
#dropping columns with sum less than 2
cols_with_less_than_25 = [col for col in X_tfidf_tagged.columns 
                                 if abs(X_tfidf_tagged[col].sum()) < 2]
reduced_X_train_tagged = X_tfidf_tagged.drop(cols_with_less_than_25, axis=1)

In [1471]:
#reduced_X_train_tagged.shape

In [1472]:
#reduced_X_train_tagged.head()

In [1473]:
from nltk.tokenize import word_tokenize as WordTokenizer

def word_tokenizer(data, col):
    token=[]
    for item in data[col]:
         token.append(WordTokenizer(item))

    return token


In [1474]:
word_token = word_tokenizer(text_df, "text")
#df.insert(index, 'token_column', token)
#print(token)
#tokenize_data = pd.DataFrame(token)
#word_token

In [1475]:
#tokenize_data.head()
#word_token

In [1476]:
def penn_to_wn(tag):
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [1477]:
import nltk
nltk.download('sentiwordnet')
nltk.download('stopwords')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/noman/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/noman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [1478]:
word_token[39]

['Be',
 'sure',
 'to',
 'accompany',
 'your',
 '<',
 'aspect',
 '>',
 'with',
 'one',
 'of',
 'their',
 'fresh',
 'juice',
 'concoctions',
 '.']

In [1479]:
stop_words=['aspect','<','>','is','the','of','for','and','.',',']
word_token_reduced=[]
for token in word_token:
    new_token =  [x for x in token if x not in stop_words]
    word_token_reduced.append(new_token)

In [1480]:
word_token_reduced[39]

['Be',
 'sure',
 'to',
 'accompany',
 'your',
 'with',
 'one',
 'their',
 'fresh',
 'juice',
 'concoctions']

In [1481]:
#polarity shifter
negWords = ['no', 'not', 'never', "n't"]
def polarityShifter(word, index, negList):
    if(word in negWords):
        negList.append(index + 1)
        negList.append(index + 2)
    
    return negList

In [1482]:
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
lemmatizer = WordNetLemmatizer()

def sentiWeights(words):
    tagged_word = []
    tagged = pos_tag(words)
    #print(tagged)
    #sentiment = 0.0
    idx=-1
    for word, tag in tagged:
        sentiment = 0.0
        negIndex = []
        prevIndex = -1
        idx = idx + 1 
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV,wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue

        synsets = wn.synsets(lemma, pos=wn_tag)
        if not synsets:
            continue

        # Take the first sense, the most common
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        negIndex = polarityShifter(word, idx, negIndex)
        sentiment += swn_synset.pos_score() - swn_synset.neg_score()
        if(idx in negIndex and prevIndex != idx - 1):
            prevIndex = idx
            sentiment *= -1
        tagged_word.append([word,sentiment,idx])
        
    return tagged_word

In [1483]:
sent_score_word = [] 
for token in word_token_reduced:
    #print(token)
    sent_score_word.append(sentiWeights(token))
#print(sent_score_word)

In [1484]:
#tagged_word
sent_score_word[0]

[['fried', 0.0, 2], ['pork', 0.0, 3], ['dumplings', 0.0, 4]]

In [1485]:
len(sent_score_word)

1120

In [1486]:
#word_token_reduced
def getScoreForSentiments(feature_columns_sentiword,sent_score_word):
    my_df= []
    for senti in sent_score_word:
        number_of_pos = 0
        number_of_neg = 0
        sum_of_scores = 0
        maximum_token_score = 0
        minimum_token_score = 0
        for se in senti:
            if se[1] > 0:
                number_of_pos = number_of_pos+1

            if se[1] < 0:
                number_of_neg = number_of_neg+1

            sum_of_scores = sum_of_scores + se[1]

            if se[1] > maximum_token_score:
                maximum_token_score = se[1]

            if se[1] < minimum_token_score:
                minimum_token_score = se[1]

        #print(number_of_pos)

        d = {
            feature_columns_sentiword[0] : number_of_pos,  
            feature_columns_sentiword[1] : number_of_neg,
            feature_columns_sentiword[2] : sum_of_scores,
            feature_columns_sentiword[3] : maximum_token_score,
            feature_columns_sentiword[4] : minimum_token_score
        }
        my_df.append(d)

    my_df = pd.DataFrame(my_df)
    return my_df

In [1487]:
#bing liu's opinion lexicon - negative words
f = open('opinion-lexicon-English/negative-words.txt', 'r',encoding = "ISO-8859-1")
negative_words_liu = f.read().splitlines()
f.close()

In [1488]:
f = open('opinion-lexicon-English/positive-words.txt', 'r',encoding = "ISO-8859-1")
positive_words_liu = f.read().splitlines()
f.close()

In [1489]:
#negative_words_liu

In [1490]:
def sentiWeightsLiu(words):
    tagged_word = []
    tagged = pos_tag(words)
    #print(tagged)
    #sentiment = 0.0
    idx=-1
    for word, tag in tagged:
        sentiment = 0.0
        negIndex = []
        prevIndex = -1
        idx = idx + 1
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV,wn.VERB):
            continue

        if word in positive_words_liu:
            sentiment = 1.0
        if word in negative_words_liu:
            sentiment = -1.0
            
        negIndex = polarityShifter(word, idx, negIndex)

        if(idx in negIndex and prevIndex != idx - 1):
            prevIndex = idx
            sentiment *= -1
            
        tagged_word.append([word,sentiment,idx])
        
    return tagged_word

In [1491]:
#assigning weights on basis of liu's opinion lexicon

sent_score_word_liu = [] 
for token in word_token_reduced:
    #print(token)
    sent_score_word_liu.append(sentiWeightsLiu(token))

In [1492]:
sent_score_word_liu[1]

[['very', 0.0, 1],
 ['sharp', 1.0, 2],
 ['look', 0.0, 4],
 ['good', 1.0, 5],
 ['too', 0.0, 6]]

In [1493]:
aspect_locations_tokens = []
for word_ in word_token:
    aspect_locations_tokens.append(word_.index("aspect")-2)

In [1494]:
sent_score_word_liu_new = []
for idx,sent_liu in enumerate(sent_score_word_liu):
    start_range = aspect_locations_tokens[idx]-3
    end_range = aspect_locations_tokens[idx]+3
    sent_new =[]
    for sent in sent_liu:
        if sent[2] >= start_range and sent[2] <= end_range:
            sent[1] = sent[1] * 2
        sent_new.append(sent)
    sent_score_word_liu_new.append(sent_new)

In [1495]:
sent_score_word_new = []
for idx,sent_senti in enumerate(sent_score_word):
    start_range = aspect_locations_tokens[idx]-3
    end_range = aspect_locations_tokens[idx]+3
    sent_new =[]
    for sent in sent_senti:
        if sent[2] >= start_range and sent[2] <= end_range:
            sent[1] = sent[1] * 2
        sent_new.append(sent)
    sent_score_word_new.append(sent_new)

In [1496]:
sent_score_word_new[0]

[['fried', 0.0, 2], ['pork', 0.0, 3], ['dumplings', 0.0, 4]]

In [1497]:
sent_score_word_liu_new[2]

[['best', 2.0, 1],
 ['dessert', 0.0, 2],
 ["n't", 0.0, 4],
 ['particularly', 0.0, 5],
 ['Hawaiian', 0.0, 6],
 ["'s", 0.0, 9],
 ['small', 0.0, 11],
 ['world', 0.0, 12],
 ['comes', 0.0, 15],
 ['sweets', 0.0, 17]]

In [1498]:
sent_score_word_liu[2]

[['best', 2.0, 1],
 ['dessert', 0.0, 2],
 ["n't", 0.0, 4],
 ['particularly', 0.0, 5],
 ['Hawaiian', 0.0, 6],
 ["'s", 0.0, 9],
 ['small', 0.0, 11],
 ['world', 0.0, 12],
 ['comes', 0.0, 15],
 ['sweets', 0.0, 17]]

In [1499]:
#mpqa lexicon
f = open('mpqa/mpqa.txt', 'r',encoding = "ISO-8859-1")
mpqa_lines = f.read().splitlines()
f.close()

In [1500]:
mpqa_sent_score_word = []
for mpqa in mpqa_lines:
    mpqas = mpqa.split(" ")
    mpqa_types = mpqas[0].split("=")
    mpqa_type = mpqa_types[1]
    mpqa_words = mpqas[2].split("=")
    mpqa_word = mpqa_words[1]
    #print(mpqas)
    #print(mpqas[5])
    mpqa_polarities = mpqas[5].split("=")
    mpqa_polarity = mpqa_polarities[1]
    score = 0.0
    if mpqa_polarity == "negative":
        if mpqa_type == "weaksubj":
            score = -0.5
        else:
            score = -1.0
            
    if mpqa_polarity == "positive":
        if mpqa_type == "weaksubj":
            score = 0.5
        else:
            score = 1.0
    
    mpqa_sent_score_word.append([mpqa_word,score])
    

In [1501]:
mpqa_sent_score_word[4]

['abasement', -1.0]

In [1502]:
for mpqa_w in mpqa_sent_score_word:
    if 'be' == mpqa_w[0]:
        print(mpqa_w[1])
        break

In [1503]:
def sentiWeightsMpqa(words):
    tagged_word = []
    tagged = pos_tag(words)
    #print(tagged)
    
    idx=-1
    for word, tag in tagged:
        sentiment = 0.0
        negIndex = []
        prevIndex = -1
        idx = idx + 1
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV,wn.VERB):
            continue

        for mpqa_w in mpqa_sent_score_word:
            if word == mpqa_w[0]:
                sentiment = mpqa_w[1]
                break
        
        negIndex = polarityShifter(word, idx, negIndex)

        if(idx in negIndex and prevIndex != idx - 1):
            prevIndex = idx
            sentiment *= -1
            
        tagged_word.append([word,sentiment,idx])
        
    return tagged_word

In [1504]:
sent_score_word_mpqa = [] 
for token in word_token_reduced:
    #print(token)
    sent_score_word_mpqa.append(sentiWeightsMpqa(token))

In [1505]:
sent_score_word_mpqa[4]

[['never', 0.0, 1],
 ['had', 0.0, 2],
 ['so', 0.0, 5],
 ['gave', 0.0, 7],
 ['shot', 0.0, 10]]

In [1506]:
# populating unigram dataframe with weighted sentiment scores of mpqa

for idx,sent_word_list in enumerate(sent_score_word_mpqa):
    for sent_word in sent_word_list:
        if sent_word[0] in uni_df.columns:
            uni_df.loc[idx,sent_word[0]] = sent_word[1]
            
    

In [1507]:
uni_df.shape

(1120, 1474)

In [1508]:
# populating unigram dataframe with weighted sentiment scores of liu

for idx,sent_word_list in enumerate(sent_score_word_liu):
    for sent_word in sent_word_list:
        if sent_word[0] in uni_df.columns:
            uni_df.loc[idx,sent_word[0]] = uni_df.loc[idx,sent_word[0]] + sent_word[1]

In [1509]:
# populating unigram dataframe with weighted sentiment scores of sentiword

for idx,sent_word_list in enumerate(sent_score_word_new):
    for sent_word in sent_word_list:
        if sent_word[0] in uni_df.columns:
            uni_df.loc[idx,sent_word[0]] = (uni_df.loc[idx,sent_word[0]] + sent_word[1])

In [1510]:
uni_df.shape

(1120, 1474)

In [1511]:
uni_df = uni_df.add_suffix('_without_tag')

In [1512]:
reduced_uni_df = reduced_uni_df.add_suffix('_without_tag')

In [1270]:
#Amazon unigram lexicon

f = open('opinion-lexicon-English/Amazon-laptops-electronics-reviews-unigrams.txt', 'r',encoding = "ISO-8859-1")
domain_specific_lex = f.read().splitlines()
f.close()

In [1513]:
#Yelp unigram lexicon
f = open('opinion-lexicon-English/Yelp-restaurant-reviews-unigrams.txt', 'r',encoding = "ISO-8859-1")
domain_specific_lex = f.read().splitlines()
f.close()

In [1514]:
lex_score_list = {}
lex_list = []
for lexes in domain_specific_lex:
        splitted_lex = lexes.split(',')
        lex_score_list[splitted_lex[0]] = splitted_lex[1]
        lex_list.append(splitted_lex[0])

In [1515]:
def sentiWeightsLex(words):
    tagged_word = []
    #tagged = pos_tag(words)
    #print(tagged)
    idx=-1
   
    prevIndex = -1
    
    #for word, tag in tagged:
        #wn_tag = penn_to_wn(tag)
        #if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV,wn.VERB):
            #continue
    for word in words:
        sentiment = 0.0
        negIndex = []
        idx = idx + 1
        if word in lex_list:
            sentiment = float(lex_score_list[word])
        
        negIndex = polarityShifter(word, idx, negIndex)

        if(idx in negIndex and prevIndex != idx - 1):
            prevIndex = idx
            sentiment *= -1
            #print(word, sentiment, idx)
            
        tagged_word.append([word,sentiment,idx])
        
    return tagged_word

In [1516]:
senti_score_lex = [] 
for token in word_token_reduced:
    senti_score_lex.append(sentiWeightsLex(token))

In [1517]:
senti_score_lex[0]

[['I', 0.0, 0],
 ['reccomend', 1.159, 1],
 ['fried', 0.255, 2],
 ['pork', 0.577, 3],
 ['dumplings', 0.522, 4],
 ['orange', 0.363, 5],
 ['chicken/beef', -0.045, 6]]

In [1518]:
#Bit of modification

def getScoreForSentimentsLex(feature_columns_sentiword,sent_score_word):
    my_df= []
    for senti in sent_score_word:
        total_pos = 0
        total_neg = 0
        sum_of_scores = 0
        maximum_token_score = 0
        minimum_token_score = 0
        for se in senti:
            if se[1] > 0:
                total_pos += se[1]

            if se[1] < 0:
                total_neg += se[1]

            sum_of_scores = sum_of_scores + se[1]

            if se[1] > maximum_token_score:
                maximum_token_score = se[1]

            if se[1] < minimum_token_score:
                minimum_token_score = se[1]

        #print(number_of_pos)

        d = {
            feature_columns_sentiword[0] : total_pos,  
            feature_columns_sentiword[1] : total_neg,
            feature_columns_sentiword[2] : sum_of_scores,
            feature_columns_sentiword[3] : maximum_token_score,
            feature_columns_sentiword[4] : minimum_token_score
        }
        my_df.append(d)

    my_df = pd.DataFrame(my_df)
    return my_df

In [1519]:
# populating unigram dataframe with weighted sentiment scores of sentiword

for idx,sent_word_list in enumerate(senti_score_lex):
    for sent_word in sent_word_list:
        if sent_word[0] in uni_df.columns:
            uni_df.loc[idx,sent_word[0]] = (uni_df.loc[idx,sent_word[0]] + sent_word[1])/4

In [1520]:
uni_df.shape

(1120, 1474)

In [1521]:
#dropping columns with absolute sum less than 0.001
cols_with_less_than_abs_sum = [col for col in uni_df.columns 
                                 if abs(uni_df[col].sum()) < 2]
reduced_uni_df = uni_df.drop(cols_with_less_than_abs_sum, axis=1)

In [1522]:
reduced_uni_df.shape

(1120, 295)

In [1523]:
feature_columns_sentiword_lex = ['total positives lex_test','total negatives lex_test','sum of scores lex_test','maximum token score lex_test','minimum token score lex_test']


my_df_lex = getScoreForSentimentsLex(feature_columns_sentiword_lex,senti_score_lex)

my_df_lex.head()

,maximum token score lex_test,minimum token score lex_test,sum of scores lex_test,total negatives lex_test,total positives lex_test
0,1.159,-0.045,2.831,-0.045,2.876
1,0.600,-0.133,1.265,-0.133,1.398
2,1.448,-0.202,4.816,-0.601,5.417
3,2.262,-0.274,3.405,-0.749,4.154
4,0.150,-0.485,-1.340,-1.526,0.186


In [1524]:
feature_columns_sentiword = ['number of positives_test','number of negatives_test','sum of scores_test','maximum token score_test','minimum token score_test']

my_df_senti = getScoreForSentiments(feature_columns_sentiword,sent_score_word_new)

In [1525]:
feature_columns_sentiword_liu = ['number of positives liu_test','number of negatives liu_test','sum of scores liu_test','maximum token score liu_test','minimum token score liu_test']


my_df_liu = getScoreForSentiments(feature_columns_sentiword_liu,sent_score_word_liu_new)

In [1526]:
feature_columns_sentiword_mpqa = ['number of positives mpqa_test','number of negatives mpqa_test','sum of scores mpqa_test','maximum token score mpqa_test','minimum token score mpqa_test']


my_df_mpqa = getScoreForSentiments(feature_columns_sentiword_mpqa,sent_score_word_mpqa)

In [1527]:
my_df_liu.shape

(1120, 5)

In [ ]:
vocab_list_senti = []


In [6]:
train_data.loc[1," term_location"]

'9--28'

In [7]:
#train_data[" text"]

In [39]:
#new_text_aspect

In [42]:
len(features)

2203

In [44]:
len(feature_main) 

2063

In [47]:
len(vocab_list_set)

5752

In [35]:
#vocab_list_set

In [166]:
#vocab_list_set

In [167]:
#cv = CountVectorizer(vocabulary=vocab_list_set)
#cv.fit_transform(train_data[" text"]).toarray()
#cv.vocabulary_

In [113]:
len(vocab_list)

23619

In [96]:
X_term_freq.shape

(2203, 5752)

In [97]:
X_term_freq.head()

,0,1,2,3,4,5,6,7,8,9,...,5742,5743,5744,5745,5746,5747,5748,5749,5750,5751
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [764]:
reduced_X_train.shape

(2203, 331)

In [122]:
#for idx,ngrm in enumerate(ngram_set):
#    print('ngram: {0}\n'.format(ngrm))
#    print('ngram.shape: {0}'.format(ngrm.shape))
#    print('vectorizer.vocabulary_: {0}'.format(vectorizers[idx].vocabulary_))

In [1529]:
train_data_mix_back = train_data_mix_back.drop(my_df_liu.columns,axis=1)
train_data_mix_back = train_data_mix_back.drop(my_df_senti.columns,axis=1)
train_data_mix_back = train_data_mix_back.drop(my_df_mpqa.columns,axis=1)
#train_data_mix_back = train_data_mix_back.drop(my_df_lex.columns,axis=1)

ValueError: labels ['maximum token score liu_test' 'minimum token score liu_test'
 'number of negatives liu_test' 'number of positives liu_test'
 'sum of scores liu_test'] not contained in axis

In [1338]:
#train_data_mix_back = train_data_mix_back.drop(my_df_senti.columns,axis=1)

In [1530]:
train_data_mix =[]
train_data_mix_back = my_df_liu.join(train_data_mix_back)
#train_data_mix = my_df_liu.join(my_df_senti)

In [1533]:
train_data_mix_back.shape

(1120, 1434)

In [1532]:
train_data_mix_back = train_data_mix_back.join(my_df_senti)

In [1297]:
#train_data_mix = train_data_mix.join(my_df_liu,lsuffix='_left', rsuffix='_right')

In [1534]:
train_data_mix_back = train_data_mix_back.join(my_df_mpqa,lsuffix='_left', rsuffix='_right')
#train_data_mix = train_data_mix.join(my_df_mpqa,lsuffix='_left', rsuffix='_right')

In [1425]:
reduced_uni_df.shape

(3602, 668)

In [1426]:

train_data_mix = reduced_uni_df.join(train_data_mix,lsuffix='_left', rsuffix='_right')

In [1427]:
train_data_mix.shape

(3602, 683)

In [1428]:
train_data_mix = reduced_X_train_tagged.join(train_data_mix,lsuffix='_left', rsuffix='_right')

In [1429]:
train_data_mix_back = train_data_mix_back.join(my_df_lex,lsuffix='_left', rsuffix='_right')

In [1430]:
train_data_mix.shape

(3602, 1439)

In [1320]:
##train_data_mix = train_data_mix.join(train_data_mix_back,lsuffix='_left', rsuffix='_right')

In [1434]:
train_data_mix.head()

,00_nn,100_cd,10_cd,15_cd,1_cd,20_cd,25_cd,2_cd,30_cd,3_cd,...,maximum token score_test,minimum token score_test,number of negatives_test,number of positives_test,sum of scores_test,maximum token score mpqa_test,minimum token score mpqa_test,number of negatives mpqa_test,number of positives mpqa_test,sum of scores mpqa_test
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.250,-1.250,1,1,-1.000,0.0,-1.0,1,0,-1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.625,-0.625,2,5,0.875,1.0,0.0,0,2,1.5
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.250,-0.625,2,3,-0.250,1.0,0.0,0,3,2.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.250,-0.625,2,2,-0.250,1.0,0.0,0,3,2.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.125,-1.250,2,2,-1.125,1.0,0.0,0,3,2.0


In [1535]:
train_data_mix_back.head()

,maximum token score liu_test,minimum token score liu_test,number of negatives liu_test,number of positives liu_test,sum of scores liu_test,00_nn,100_cd,10_cd,15_cd,1_cd,...,maximum token score_test,minimum token score_test,number of negatives_test,number of positives_test,sum of scores_test,maximum token score mpqa_test,minimum token score mpqa_test,number of negatives mpqa_test,number of positives mpqa_test,sum of scores mpqa_test
0,0.0,-1.0,1,0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000,0,0,0.000,0.0,0.0,0,0,0.0
1,2.0,0.0,0,2,3.0,0.0,0.0,0.0,0.0,0.0,...,0.75,-0.125,1,2,1.125,0.5,-0.5,2,1,-0.5
2,2.0,0.0,0,1,2.0,0.0,0.0,0.0,0.0,0.0,...,1.50,-0.375,1,1,1.125,1.0,0.0,0,1,1.0
3,1.0,0.0,0,2,2.0,0.0,0.0,0.0,0.0,0.0,...,0.75,0.000,0,5,1.500,0.5,0.0,0,2,1.0
4,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.50,-1.250,2,1,-0.875,0.0,0.0,0,0,0.0


In [ ]:
##train_data_mix = train_data_mix_back.join(train_data_mix_back,lsuffix='_left', rsuffix='_right')

In [945]:
#result = pd.concat([train_data_mix_back,train_data_mix],axis=0,join='inner')

#result = train_data_mix_back.append(train_data_mix.reindex_axis(train_data_mix_back.columns, 1, fill_value=0))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [1435]:
train_data_mix.shape

(3602, 1439)

In [1436]:
Y.shape

(2203,)

In [1432]:
train_data_mix_back = train_data_mix

In [1441]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_Y, val_Y = train_test_split(train_data_mix, Y,random_state = 0)
#example_ids = val_X[" example_id"]

In [1442]:
#Random Forest train
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=683, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
forest_model.fit(train_X,train_Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=683, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [508]:
forest_model.classes_

array([-1,  0,  1])

In [1536]:
#Random Forest predict
from sklearn.metrics import accuracy_score
forest_pred = forest_model.predict(train_data_mix_back)
genOutput('Noman_Mulla_Sylvester_Raj_Data-2.txt',example_ids,forest_pred)
#accuracy_score(val_Y, forest_pred)

In [113]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
#Random Forest using cross validation and confusion matrix
forest_model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=683, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
predicted = cross_val_predict(forest_model, train_data_mix, Y, cv=10)
conf_mat_forest = confusion_matrix(Y,predicted,labels=[-1,0,1])
#calclulateF1(conf_mat_forest)
#print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))
print("Overal Accuracy")
print(accuracy_score(Y, predicted))

(array([0.6864813 , 0.68823529, 0.78080808]), array([0.8647343 , 0.26834862, 0.82321619]), array([0.76536611, 0.38613861, 0.80145153]), array([828, 436, 939]))
Overal Accuracy
0.7290059010440308


In [338]:
from xgboost import XGBClassifier


xgboost = XGBClassifier(learning_rate= 0.2, n_estimators= 300,max_depth=30)
xgboost.fit(train_X,train_Y)
xgboost_pred = xgboost.predict(val_X)
accuracy_score(val_Y, xgboost_pred)

KeyboardInterrupt: 

In [237]:
from sklearn.ensemble import BaggingClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
clf1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=99999,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=99, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)
clf2 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=683, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
clf3 = MLPClassifier()

vote_model = VotingClassifier(estimators=[('bag', clf1), ('rf', clf2), ('mlp', clf3)], voting='hard')
vote_model.fit(train_data_mix,Y)




VotingClassifier(estimators=[('bag', BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=99999,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
 ...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [503]:
vote_pred = vote_model.predict(train_data_mix)
genOutput('Noman_Mulla_Sylvester_Raj_Data-1.txt',example_ids,vote_pred)

ValueError: Number of features of the model must match the input. Model n_features is 1364 and input n_features is 543.

In [463]:
clf1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=99999,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=49, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)
clf2 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=683, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
clf3 = MLPClassifier()
vote_model = VotingClassifier(estimators=[('bag', clf1), ('rf', clf2), ('mlp', clf3)], voting='hard')
predicted = cross_val_predict(vote_model, train_data_mix, Y, cv=10)
conf_mat_forest = confusion_matrix(Y,predicted,labels=[-1,0,1])
#calclulateF1(conf_mat_forest)
#print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))
print("Overal Accuracy")
print(accuracy_score(Y, predicted))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

(array([0.7110885 , 0.62992126, 0.78674948]), array([0.8442029 , 0.36697248, 0.80937167]), array([0.7719492 , 0.46376812, 0.79790026]), array([828, 436, 939]))
Overal Accuracy
0.7349069450748978


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [120]:
# svm grid search hyper parameter tuning

from sklearn import svm, grid_search
from sklearn.model_selection import GridSearchCV
def tuneParameters(train_data_mix_fin_n,train_Y):
        parameter_candidates = [

        
        {'C': [1.566,1.691,1.7,1.9,3.56,8,10,15], 'gamma': [0.001,0.00916,0.00888,0.009,0.009785,0.01,1,3],'kernel': ['rbf']},
            ]

        # Create a classifier object with the classifier and parameter candidates
        clf = GridSearchCV(estimator=svm.SVC(), param_grid=parameter_candidates,cv=10)

        # Train the classifier on data1's feature and target data
        clf.fit(train_data_mix_fin_n, train_Y)
        # View the accuracy score
        print('Best score for data:', clf.best_score_) 

        # View the best parameters for the model found using grid search
        print('Best C:',clf.best_estimator_.C) 
        best_C = clf.best_estimator_.C
        print('Best Kernel:',clf.best_estimator_.kernel)
        best_kernel = clf.best_estimator_.kernel
        print('Best Gamma:',clf.best_estimator_.gamma)
        best_gamma = clf.best_estimator_.gamma
        print('Best Degree:',clf.best_estimator_.degree)
        best_degree = clf.best_estimator_.degree
        return best_C,best_kernel,best_gamma,best_degree

In [121]:
tuneParameters(train_data_mix,Y)

Best score for data: 0.7172038129822969
Best C: 15
Best Kernel: rbf
Best Gamma: 0.01
Best Degree: 3


(15, 'rbf', 0.01, 3)

In [238]:
#Import Library
from sklearn import svm
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create SVM classification object 
# C = [3.1,3.5,3.55,3.981,3.764,3.333]
# gamma = [0.01,0.03,0.05,0.055,0.066,0.077,0.091,0.092,0.0933,0.1]
model = svm.SVC(kernel='rbf', C=15, gamma=0.01) 
# there is various option associated with it, like changing kernel, gamma and C value. Will discuss more # about it in next section.Train the model using the training sets and check score
model.fit(train_X, train_Y)
print(model.score(train_X, train_Y))
#Predict Output
predicted= model.predict(val_X)
accuracy_score(val_Y, predicted)

0.8849878934624698


0.7041742286751361

In [127]:
svm_model = svm.SVC(kernel='rbf',  C=15, gamma=0.015) 
predicted = cross_val_predict(svm_model, train_data_mix, Y, cv=10)
conf_mat_forest = confusion_matrix(Y,predicted,labels=[-1,0,1])
#calclulateF1(conf_mat_forest)
#print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))
print("Overal Accuracy")
print(accuracy_score(Y, predicted))

(array([0.71012931, 0.5613079 , 0.79955947]), array([0.79589372, 0.47247706, 0.77316294]), array([0.75056948, 0.51307597, 0.78613969]), array([828, 436, 939]))
Overal Accuracy
0.7221970040853382


In [1202]:
def genOutput(output_file, example_ids, predicted):
    output_file = open(output_file,'w')
    for i in range(0, len(predicted)):
        output_file.write(str(example_ids[i]) +";;"+ str(predicted[i])+"\n")
        #print(predicted[i])